<header style="padding:1px;background:#f9f9f9;border-top:3px solid #00b2b1"><img id="Teradata-logo" src="https://www.teradata.com/Teradata/Images/Rebrand/Teradata_logo-two_color.png" alt="Teradata" width="220" align="right" />

<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>Telco Customer Churn</b>
</header>

<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>Introduction</b></p>

<p style = 'font-size:16px;font-family:Arial'>
Telco customer churn or customer attrition refers to the phenomenon where customers of a telecommunications company (telco)
cancel or stop using their services, resulting in a loss of revenue for the telco. Keeping the customers with them is significant for the company as getting new customers is more resource and time consuming than retaining the old customers. Customers may churn due to various reasons like dissatisfaction with service quality, pricing, customer service, or finding better alternatives from competitors. To identify the factors contributing to the churn and to take appropriate measure to retain customers. Telco companies analyze large amounts of large amounts of customer data, such as usage patterns, billing information, demographics, and interactions, to find patterns that may indicate customers who are at risk of churning.</p>

<p style = 'font-size:16px;font-family:Arial'>
Machine learning and predictive analytics are commonly used to build models that can predict customers which are likely to churn in future. Once customers at risk of churning are identified, telecom companies can take actions to reduce churn. This may include targeted marketing campaigns, personalized offers, improved customer service, or addressing customer concerns. By being proactive in addressing customer issues and offering incentives to stay, telco companies can improve customer retention rates and reduce churn. </p>

<p style = 'font-size:16px;font-family:Arial'>
Managing telco churn is complex and requires continuous monitoring, analysis, and proactive customer engagement strategies. By using data and advanced analytics, telecom companies can better understand customer behavior and preferences, and take proactive measures to retain customers and maintain profitability.</p>

<p style = 'font-size:16px;font-family:Arial'>
Let's demonstrate this use case with sample data using InDb analytics in Vantage which can pre-process and analyze huge amounts of data and at scale.   
</p>

<h1 style = 'font-size:28px;font-family:Arial;color:#E37C4D'><b>Connect to Vantage, Import python packages and explore the dataset</b></h1>


In [ ]:
#import libraries
import matplotlib.pyplot as plt 
import matplotlib.ticker as mtick 
import pandas as pd
import numpy as np
import getpass
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=DeprecationWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)

from teradataml import *

import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import seaborn as sns

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
display.max_rows=5

<p style = 'font-size:16px;font-family:Arial'>You will be prompted to provide the password. Enter your password, press the Enter key, then <b>use down arrow</b> to go to next cell.</p>

In [ ]:
%run -i ../startup.ipynb
eng = create_context(host = 'host.docker.internal', username='demo_user', password = password)
print(eng)

In [ ]:
%%capture
execute_sql('''SET query_band='DEMO=Telco_Customer_Churn_PY_SQL.ipynb;' UPDATE FOR SESSION; ''')

<h2 style = 'font-size:20px;font-family:Arial;color:#E37C4D'> <b>Getting Data for This Demo</b></h2>
<p style = 'font-size:16px;font-family:Arial'>We have provided data for this demo on cloud storage. You have the option of either running the demo using foreign tables to access the data without using any storage on your environment or downloading the data to local storage which may yield somewhat faster execution, but there could be considerations of available storage. There are two statements in the following cell, and one is commented out. You may switch which mode you choose by changing the comment string.</p>

In [ ]:
%run -i ../run_procedure.py "call get_data('DEMO_Telco_cloud');"
 # takes about 30 seconds, estimated space: 0 MB
#%run -i ../run_procedure.py "call get_data('DEMO_Telco_local');" 
# takes about 1 minute 30 seconds, estimated space: 4 MB

<p style = 'font-size:16px;font-family:Arial'>Next is an optional step – if you want to see status of databases/tables created and space used.</p>

In [ ]:
%run -i ../run_procedure.py "call space_report();"

<h1 style = 'font-size:28px;font-family:Arial;color:#E37C4D'><b>Data Exploration</b></h1>

<h2 style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>Customer Churn</b></h2>
<p style = 'font-size:16px;font-family:Arial'>Create a "Virtual DataFrame" that points to the data set in Vantage. Check the shape of the dataframe as check the datatypes of all the columns of the dataframe.</p>

In [ ]:
tdf = DataFrame(in_schema("DEMO_Telco", "Customer_Churn"))
tdf

<p style = 'font-size:16px;font-family:Arial'>Check the demographics of data

In [ ]:
print("Shape of the data: ", tdf.shape)
tdf.info()

<p style = 'font-size:16px;font-family:Arial'> As we can see from shape and info methods our dataset has 7043 rows with 21 columns

<p style = 'font-size:16px;font-family:Arial;color:#E37C4D'><b>Summary of columns</b><br>
<p style = 'font-size:16px;font-family:Arial;'>The <b>ColumnSummary</b> function can be used to take a quick look at the columns, their datatypes, and summary of NULLs/non-NULLs for a given table.   

In [ ]:
from teradataml import ColumnSummary
obj = ColumnSummary(data=tdf,
                        target_columns=[':']
                       )

In [ ]:
obj.result.head(21)

<h1 style = 'font-size:28px;font-family:Arial;color:#E37C4D'><b>Exploratory Data Analysis</b></h1>

<p style = 'font-size:16px;font-family:Arial'>
Exploratory Data Analysis (EDA) refers to the process of visually and statistically examining, analysing, and summarizing data to understand its characteristics, patterns, and relationships.<br>As Vantage is not a visualization tool we will use Python libraries for visualization. Here we can see how seamless it is to use Teradata Vantage with python.

In [ ]:
#converting teradata dataframe to pandas dataframe for visualization
df=tdf.to_pandas().reset_index()

In [ ]:
#customerid is unique for each row hence don't have any effect on the predictions
df = df.drop(['CustomerID'], axis = 1 )
#df = df.drop(['CustomerID'])
df.head()

In [ ]:
#Gender and Churn percentage distribution
g_labels = ['Male', 'Female']
c_labels = ['No', 'Yes']
# Create subplots: use 'domain' type for Pie subplot
fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]])
fig.add_trace(go.Pie(labels=g_labels, values=df['Gender'].value_counts(), name="Gender"),
              1, 1)
fig.add_trace(go.Pie(labels=c_labels, values=df['Churn'].value_counts(), name="Churn"),
              1, 2)

# Use `hole` to create a donut-like pie chart
fig.update_traces(hole=.4, hoverinfo="label+percent+name", textfont_size=16)

fig.update_layout(
    title_text="Gender and Churn Distributions",
    # Add annotations in the center of the donut pies.
    annotations=[dict(text='Gender', x=0.16, y=0.5, font_size=20, showarrow=False),
                 dict(text='Churn', x=0.84, y=0.5, font_size=20, showarrow=False)])
fig.show()

<p style = 'font-size:16px;font-family:Arial'>26.6 % of customers switched to another firm.<br>
Customers are 49.5 % female and 50.5 % male.

In [ ]:
#Churn per Gender
plt.figure(figsize=(6, 6))
labels =["Churn: Yes","Churn:No"]
values = [1869,5163]
labels_gender = ["F","M","F","M"]
sizes_gender = [939,930 , 2544,2619]
colors = ['#ff6666', '#66b3ff']
colors_gender = ['#c2c2f0','#ffb3e6', '#c2c2f0','#ffb3e6']
explode = (0.3,0.3) 
explode_gender = (0.1,0.1,0.1,0.1)
textprops = {"fontsize":15}
#Plot
plt.pie(values, labels=labels,autopct='%1.1f%%',pctdistance=1.08, labeldistance=0.8,colors=colors, startangle=90,frame=True, explode=explode,radius=10, textprops =textprops, counterclock = True, )
plt.pie(sizes_gender,labels=labels_gender,colors=colors_gender,startangle=90, explode=explode_gender,radius=7, textprops =textprops, counterclock = True, )
#Draw circle
centre_circle = plt.Circle((0,0),5,color='black', fc='white',linewidth=0)
fig = plt.gcf()
fig.gca().add_artist(centre_circle)

plt.title('Churn Distribution w.r.t Gender: Male(M), Female(F)', fontsize=15, y=1.1)

# show plot 
 
plt.axis('equal')
plt.tight_layout()
plt.show()

<p style = 'font-size:16px;font-family:Arial'>There is negligible difference in customer percentage/ count who changed the service provider. Both genders behaved in similar fashion when it comes to migrating to another service provider.

In [ ]:
fig = px.histogram(df, x="Churn", color="Contract", barmode="group", title="<b>Customer contract distribution<b>")
fig.update_layout(width=700, height=500, bargap=0.1)
fig.show()

<p style = 'font-size:16px;font-family:Arial'> About 75% of customer with Month-to-Month Contract opted to move out as compared to 13% of customers with One Year Contract and 3% with Two Year Contract

In [ ]:
labels = df['PaymentMethod'].unique()
values = df['PaymentMethod'].value_counts()

fig = go.Figure(data=[go.Pie(labels=labels, values=values, hole=.3)])
fig.update_layout(title_text="<b>Payment Method Distribution</b>")
fig.show()

In [ ]:
fig = px.histogram(df, x="Churn", color="PaymentMethod", title="<b>Customer Payment Method distribution w.r.t. Churn</b>")
fig.update_layout(width=700, height=500, bargap=0.1)
fig.show()

<p style = 'font-size:16px;font-family:Arial'>Major customers who moved out were having Electronic Check as Payment Method.
<br>Customers who opted for Credit-Card automatic transfer or Bank Automatic Transfer and Mailed Check as Payment Method were less likely to move out.  

In [ ]:
fig = go.Figure()

fig.add_trace(go.Bar(
  x = [['Churn:No', 'Churn:No', 'Churn:Yes', 'Churn:Yes'],
       ["Female", "Male", "Female", "Male"]],
  y = [965, 992, 219, 240],
  name = 'DSL',
))

fig.add_trace(go.Bar(
  x = [['Churn:No', 'Churn:No', 'Churn:Yes', 'Churn:Yes'],
       ["Female", "Male", "Female", "Male"]],
  y = [889, 910, 664, 633],
  name = 'Fiber optic',
))

fig.add_trace(go.Bar(
  x = [['Churn:No', 'Churn:No', 'Churn:Yes', 'Churn:Yes'],
       ["Female", "Male", "Female", "Male"]],
  y = [690, 717, 56, 57],
  name = 'No Internet',
))

fig.update_layout(title_text="<b>Churn Distribution w.r.t. Internet Service and Gender</b>")

fig.show()

<p style = 'font-size:16px;font-family:Arial'> A lot of customers choose the Fiber optic service and it's also evident that the customers who use Fiber optic have high churn rate, this might suggest a dissatisfaction with this type of internet service.
<br> Customers having DSL service are majority in number and have less churn rate compared to Fibre optic service.

In [ ]:
color_map = {"Yes": "#FF97FF", "No": "#AB63FA"}
fig = px.histogram(df, x="Churn", color="Dependents", barmode="group", title="<b>Dependents distribution</b>", color_discrete_map=color_map)
fig.update_layout(width=700, height=500, bargap=0.1)
fig.show()

<p style = 'font-size:16px;font-family:Arial'>Customers without dependents are more likely to churn

In [ ]:
color_map = {"Yes": '#FFA15A', "No": '#00CC96'}
fig = px.histogram(df, x="Churn", color="Partner", barmode="group", title="<b>Chrun distribution w.r.t. Partners</b>", color_discrete_map=color_map)
fig.update_layout(width=700, height=500, bargap=0.1)
fig.show()

<p style = 'font-size:16px;font-family:Arial'>Customers that don't have partners are more likely to churn

In [ ]:
color_map = {"Yes": '#FFA15A', "No": '#00CC96'}
fig = px.histogram(df, x="Churn", color="PaperlessBilling",  title="<b>Chrun distribution w.r.t. Paperless Billing</b>", color_discrete_map=color_map)
fig.update_layout(width=700, height=500, bargap=0.1)
fig.show()

<p style = 'font-size:16px;font-family:Arial'>Customers with Paperless Billing are most likely to churn.

In [ ]:
sns.set_context("paper",font_scale=1.1)
ax = sns.kdeplot(df.MonthlyCharges[(df["Churn"] == 'No') ],
                color="Red", shade = True);
ax = sns.kdeplot(df.MonthlyCharges[(df["Churn"] == 'Yes') ],
                ax =ax, color="Blue", shade= True);
ax.legend(["Not Churn","Churn"],loc='upper right');
ax.set_ylabel('Density');
ax.set_xlabel('Monthly Charges');
ax.set_title('Distribution of monthly charges by churn');


<p style = 'font-size:16px;font-family:Arial'>Customers with higher Monthly Charges are also more likely to churn

In [ ]:
ax = sns.kdeplot(df.TotalCharges[(df["Churn"] == 'No') ],
                color="Gold", shade = True);
ax = sns.kdeplot(df.TotalCharges[(df["Churn"] == 'Yes') ],
                ax =ax, color="Green", shade= True);
ax.legend(["Not Churn","Churn"],loc='upper right');
ax.set_ylabel('Density');
ax.set_xlabel('Total Charges');
ax.set_title('Distribution of total charges by churn');

In [ ]:
fig = px.box(df, x='Churn', y = 'Tenure')

# Update yaxis properties
fig.update_yaxes(title_text='Tenure (Months)', row=1, col=1)
# Update xaxis properties
fig.update_xaxes(title_text='Churn', row=1, col=1)

# Update size and title
fig.update_layout(autosize=True, width=750, height=600,
    title_font=dict(size=25, family='Courier'),
    title='<b>Tenure vs Churn</b>',
)

fig.show()

<p style = 'font-size:16px;font-family:Arial'>New customers are more likely to churn

In [ ]:
plt.figure(figsize=(25, 10))

corr = df.apply(lambda x: pd.factorize(x)[0]).corr()

mask = np.triu(np.ones_like(corr, dtype=bool))

ax = sns.heatmap(corr, mask=mask, xticklabels=corr.columns, yticklabels=corr.columns, annot=True, linewidths=.2, cmap='coolwarm', vmin=-1, vmax=1)

<p style = 'font-size:16px;font-family:Arial'>The above chart shows the heatmap of correlation of all the attributes with each other.

<h1 style = 'font-size:28px;font-family:Arial;color:#E37C4D'><b>Data Preprocessing</b></h1>

 <p style = 'font-size:16px;font-family:Arial'>Before the data can be used for model creation; we will need to do some data cleansing and transformation on it. We can do this InDb with Teradata Vantage's inbuilt functions.<br>The <b>CategoricalSummary</b> function displays the distinct values and their counts for each specified input DataFrame column.

In [ ]:
from teradataml import CategoricalSummary
CatSum = CategoricalSummary(data=tdf,target_columns=["MultipleLines","InternetService","OnlineSecurity","OnlineBackup","DeviceProtection","TechSupport","StreamingTV","StreamingMovies"])
CatSum.result.sort("ColumnName")

<p style = 'font-size:16px;font-family:Arial'>
As we can see from the sample data above and the categorical summary values, the columns 
<ul style = 'font-size:16px;font-family:Arial'><li>OnlineSecurity </li>  
<li>OnlineBackup</li>     
<li>DeviceProtection</li> 
<li>TechSupport</li>      
<li>StreamingTV</li>      
<li>StreamingMovies</li>
</ul><p style = 'font-size:16px;font-family:Arial'>are related to InternetService, wherever InternetService value is "No" the column have value of "No internet service". For our model let us replace "No internet service" to No in our  column. We will do similar operation for replacing "No phone service" to "No".

In [ ]:
from sqlalchemy import func


tdf = tdf.assign(oreplace_MultipleLines=func.oreplace(tdf.MultipleLines.expression, "No phone service","No"),
                oreplace_OnlineSecurity=func.oreplace(tdf.OnlineSecurity.expression, "No internet service","No"),
                oreplace_OnlineBackup=func.oreplace(tdf.OnlineBackup.expression, "No internet service","No"),
                oreplace_DeviceProtection=func.oreplace(tdf.DeviceProtection.expression, "No internet service","No"),                     oreplace_TechSupport=func.oreplace(tdf.TechSupport.expression, "No internet service","No"),
                oreplace_StreamingTV=func.oreplace(tdf.StreamingTV.expression, "No internet service","No"),
                oreplace_StreamingMovies=func.oreplace(tdf.StreamingMovies.expression, "No internet service","No"))
tdf

In [ ]:
# now lets drop the extra columns, rename the columns in dataframe
from teradataml.dataframe.sql_functions import case

tdf2 = tdf.assign(drop_columns=True
                ,CustomerID=tdf.CustomerID  
                ,Gender=tdf.Gender 
                ,SeniorCitizen=tdf.SeniorCitizen
                ,Partner=tdf.Partner
                ,Dependents=tdf.Dependents
                ,Tenure=tdf.Tenure
                ,PhoneService=tdf.PhoneService      
                ,MultipleLines=tdf.oreplace_MultipleLines     
                ,InternetService=tdf.InternetService     
                ,OnlineSecurity=tdf.oreplace_OnlineSecurity      
                ,OnlineBackup=tdf.oreplace_OnlineBackup        
                ,DeviceProtection=tdf.oreplace_DeviceProtection    
                ,TechSupport=tdf.oreplace_TechSupport         
                ,StreamingTV=tdf.oreplace_StreamingTV         
                ,StreamingMovies=tdf.oreplace_StreamingMovies     
                ,Contract=tdf.Contract            
                ,PaperlessBilling=tdf.PaperlessBilling    
                ,PaymentMethod=tdf.PaymentMethod       
                ,MonthlyCharges=tdf.MonthlyCharges      
                ,TotalCharges=tdf.TotalCharges        
                ,Churn = case({ "Yes" : 1, "No" : 0},value=tdf.Churn,else_=0) ) 

In [ ]:
tdf2

In [ ]:
#copying the intermediate table to database
tdf2.to_sql("churn", if_exists="replace")

<p style = 'font-size:16px;font-family:Arial;color:#E37C4D'><b>Onehotencoding & Ordinal encoding</b>.</p> 
<p style = 'font-size:16px;font-family:Arial'><b>TD_OneHotEncodingFit and Transform </b>and <b>TD_OrdinalEncodingFit and Transform</b> are Teradata's function to convert the categorical attributes to numerical.

In [ ]:
#create fit table for onehotencoding
query = '''
CREATE MULTISET VOLATILE TABLE onehotencodingfittable AS (
SELECT * FROM TD_OneHotEncodingFit (
 ON Churn AS InputTable
 USING
  TargetColumn ('Gender','Partner','Dependents','PhoneService','MultipleLines','OnlineSecurity','OnlineBackup',
  'DeviceProtection','TechSupport','StreamingTV','StreamingMovies','PaperlessBilling'
  )
  IsInputDense ('true')
  CategoryCounts(2,2,2,2,2,2,2,2,2,2,2,2)
  Approach('Auto')    
 ) AS dt
) WITH DATA
ON COMMIT PRESERVE ROWS;
'''

execute_sql(query)

<p style = 'font-size:16px;font-family:Arial'>
The other categorical columns  
<ul style = 'font-size:16px;font-family:Arial'>
<li>InternetService </li>  
<li>Contract</li>     
<li>PaperlessBilling</li> 
<li>PaymentMethod</li>      
</ul><p style = 'font-size:16px;font-family:Arial'>have more values where we can apply ordinalencoding on it         
      
         

In [ ]:
#create fit table for ordinalencoding
query = '''
SELECT * FROM TD_OrdinalEncodingFit (
ON Churn AS InputTable
OUT volatile table outputtable (ordinalencodingfittable)
USING
  TargetColumn ('InternetService','Contract','PaperlessBilling','PaymentMethod')
  DefaultValue (-1)
) as dt;
'''

execute_sql(query)

<p style = 'font-size:16px;font-family:Arial;color:#E37C4D'><b>Scale the numerical values</b><p style = 'font-size:16px;font-family:Arial'><b>TD_ScaleFit and TDScaleTransform </b>scales specified input table columns i.e perform the specific scale methods like standard deviation, mean etc to the input columns

In [ ]:
#create fit table for scale function
query = '''
select * from TD_scaleFit(
on Churn  as InputTable
OUT VOLATILE TABLE OutputTable(scaleFitOut)
using
TargetColumns('MonthlyCharges','TotalCharges')
MissValue('Keep')
ScaleMethod('range')
GlobalScale('f')
)as dt;
'''

execute_sql(query)

<p style = 'font-size:16px;font-family:Arial;color:#E37C4D'><b>Putting it altogether</b><p style = 'font-size:16px;font-family:Arial'>We will use <b> TD_ColumnTransformer</b> function to apply all the transformations from the fit tables created below in one go.

In [ ]:
query = '''
Create multiset table Transformed_data as(
SELECT CustomerId
,Churn
,SeniorCitizen
,Tenure
,InternetService
,Contract
,PaperlessBilling
,PaymentMethod
,MonthlyCharges
,TotalCharges
,Gender_0
,Gender_1
,Partner_0
,Partner_1
,Dependents_0
,Dependents_1
,PhoneService_0
,PhoneService_1
,MultipleLines_0
,MultipleLines_1
,OnlineSecurity_0
,OnlineSecurity_1
,OnlineBackup_0
,OnlineBackup_1
,DeviceProtection_0
,DeviceProtection_1
,TechSupport_0
,TechSupport_1
,StreamingTV_0
,StreamingTV_1
,StreamingMovies_0
,StreamingMovies_1
,PaperlessBilling_0
,PaperlessBilling_1	
 FROM TD_ColumnTransformer(
 ON Churn AS inputtable
 ON onehotencodingfittable AS Onehotencodingfittable DIMENSION
 ON ordinalencodingfittable AS OrdinalEncodingFitTable DIMENSION
 ON scaleFitOut AS ScaleFitTable DIMENSION         
)
AS dt
) WITH DATA
;
'''

execute_sql(query)

<p style = 'font-size:16px;font-family:Arial;color:#E37C4D'><b>Create train and test data</b><p style = 'font-size:16px;font-family:Arial'><b>TD_TrainTestSplit</b> helps us to divide our data in train and test sets for model training and scoring.

In [ ]:
query = '''Create multiset table TrainTestSplit_output as (
SELECT * FROM TD_TrainTestSplit(
ON Transformed_data AS InputTable
USING
IDColumn('CustomerId')
Seed(21)
)AS dt
) with data;
'''
try:
    execute_sql(query)
except:
    db_drop_table('TrainTestSplit_output')
    execute_sql(query)

In [ ]:
df= DataFrame("TrainTestSplit_output")
# Split into 2 virtual dataframes
df_train = df[df.TD_IsTrainRow==1].drop(["TD_IsTrainRow"], axis=1)
df_test = df[df.TD_IsTrainRow==0].drop(["TD_IsTrainRow"], axis=1)

<p style = 'font-size:16px;font-family:Arial'>We have done our preprocessing of data and we created our training and test datasets, let's now create some predictive models.

<h1 style = 'font-size:28px;font-family:Arial;color:#E37C4D'><b>InDb Model Training and Scoring</b></h1>

<h2 style = 'font-size:24px;font-family:Arial;color:#E37C4D'><b>Logistic Regression</b></h2>

<p style = 'font-size:16px;font-family:Arial'>
    <b>Logistic regression</b> is a statistical algorithm used for binary classification problems. It is a type of supervised learning algorithm that predicts the probability of an input belonging to a certain class (e.g., positive or negative) based on its features.<br>Logistic regression works by modeling the relationship between the input features and the probability of belonging to a certain class using a logistic function. The logistic function takes the input feature values and maps them onto a probability scale between 0 and 1, which represents the probability of belonging to the positive class.<br>
    The <b>TD_GLM </b>function is a generalized linear model (GLM) that performs regression and classification analysis on data sets.
<br>Please refer <a href ='https://docs.teradata.com/r/Enterprise/Teradata-Package-for-Python-Function-Reference-17.20/teradataml-Analytic-Database-SQL-Engine-Analytic-Functions/Supported-on-Database-Version-17.20.xx/MODEL-TRAINING-functions/GLM'>TD_GLM</a> for function elements and output.

In [ ]:
from teradataml import GLM, TDGLMPredict

glm_model = GLM(data = df_train,
                input_columns = ['3:'], 
                response_column = 'Churn',
                family = 'Binomial')

In [ ]:
glm_model.result

<p style = 'font-size:16px;font-family:Arial'>We have created our model, let's do the predictions on the test dataset.

In [ ]:
glm_prediction = TDGLMPredict(newdata = df_test,
                           id_column = 'CustomerID',
                           object = glm_model.result,
                           accumulate = 'Churn',
                           output_prob=True,
                           output_responses = ['0', '1'])

In [ ]:
glm_prediction.result

<p style = 'font-size:16px;font-family:Arial'>The output above shows prob_1, i.e. customer will Churn and prob_0, i.e. customer will not Churn. The prediction column uses these probabilities to give a class label, i.e. prediction column.</p>

<h3 style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>Visualize the results (ROC curve and AUC) for Logitic Regression Model</b></h3>
<p style = 'font-size:16px;font-family:Arial'>Calculate mean absolute error and AUC(Area Under the Curve) for Receiver Operating Characteristic Curve</p>
<p style = 'font-size:16px;font-family:Arial'>Mean Absolute Error is the summation of the difference between actual and predicted values averaged over the number of observations.</p>

In [ ]:
glm_pred = glm_prediction.result.to_pandas()

In [ ]:
print(mean_absolute_error(glm_pred['Churn'], glm_pred['prob_1']))

<p style = 'font-size:16px;font-family:Arial'>The ROC curve is a graph between TPR(True Positive Rate) and FPR(False Positive Rate). The area under the ROC curve is a metric of how well the model can distinguish between positive and negative classes. The higher the AUC, the better the model's performance in distinguishing between the positive and negative classes.</p>

In [ ]:
AUC = roc_auc_score(glm_pred['Churn'], glm_pred['prob_1'])
AUC

In [ ]:
fpr, tpr, thresholds = roc_curve(glm_pred['Churn'], glm_pred['prob_1'])
plt.plot(fpr, tpr, color='orange', label='ROC. AUC = {}'.format(str(AUC)))
plt.plot([0, 1], [0, 1], color='darkblue', linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend()
plt.show()

<h2 style = 'font-size:24px;font-family:Arial;color:#E37C4D'><b>XGB</b></h2>
<p style = 'font-size:16px;font-family:Arial'>
    <b>XGBoost (eXtreme Gradient Boosting) </b> is based on the gradient boosting framework, which is an ensemble learning method that combines multiple weak or base models (typically decision trees) to create a more accurate and robust predictive model. XGBoost improves upon traditional gradient boosting by using a number of optimization techniques, including parallelization, regularization, and efficient handling of missing values, to achieve faster training times and better model performance.<br>
    The <b>TD_XGBoost </b>is an implementation of the gradient boosted decision tree designed for speed and performance. In gradient boosting, each iteration fits a model to the residuals (errors) of the previous iteration to correct the errors made by existing models. The predicted residual is multiplied by this learning rate and then added to the previous prediction. Models are added sequentially until no further improvements can be made. It is called gradient boosting because it uses a gradient descent algorithm to minimize the loss when adding new models.
<br>Please refer <a href ='https://docs.teradata.com/r/Enterprise_IntelliFlex_VMware/Teradata-VantageTM-Analytics-Database-Analytic-Functions-17.20/Model-Training-Functions/TD_XGBoost'>TD_XGBoost</a> for function elements and output.

In [ ]:
query = '''CREATE MULTISET TABLE xgb_model as (
SELECT * FROM TD_XGBoost (
 ON (select * from TrainTestSplit_output where TD_IstrainRow = 1) PARTITION BY ANY 
 OUT  VOLATILE TABLE MetaInformationTable(xgb_out)
 Using
 InputColumns ('[4:]')
 ResponseColumn('Churn')
 ModelType ('classification') 
 ) as dt) with data;
'''
try:
    execute_sql(query)
except:
    db_drop_table('xgb_model')
    db_drop_table('xgb_out')
    execute_sql(query)

<p style = 'font-size:16px;font-family:Arial'>We have created our model, let's do the predictions on the test dataset.

In [ ]:
query = '''CREATE MULTISET TABLE xgb_predict_out as (
SELECT * FROM TD_XGBoostPredict(
ON (select * from TrainTestSplit_output where TD_IstrainRow = 0)  as inputtable partition by ANY
ON xgb_model as modeltable dimension order by task_index, tree_num, iter, class_num, tree_order
USING
 IdColumn('CustomerId')
 ModelType('Classification')
 OutputProb('t')
 Responses('0','1')
 accumulate('Churn')
) as dt
) with data;
'''
try:
    execute_sql(query)
except:
    db_drop_table('xgb_predict_out')
    execute_sql(query)

In [ ]:
xgb_result = DataFrame('xgb_predict_out')
xgb_result

<h3 style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>Visualize the results (ROC curve and AUC) for XGB Model</b></h3>

In [ ]:
xgb_pred = xgb_result.to_pandas().reset_index().sort_values("CustomerID")

In [ ]:
xgb_pred.head

In [ ]:
print(mean_absolute_error(xgb_pred['Churn'], xgb_pred['Prob_1']))

In [ ]:
AUC = roc_auc_score(xgb_pred['Churn'], xgb_pred['Prob_1'])
AUC

In [ ]:
fpr, tpr, thresholds = roc_curve(xgb_pred['Churn'], xgb_pred['Prob_1'])
plt.plot(fpr, tpr, color='orange', label='ROC. AUC = {}'.format(str(AUC)))
plt.plot([0, 1], [0, 1], color='darkblue', linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend()
plt.show()

<h1 style = 'font-size:28px;font-family:Arial;color:#E37C4D'><b>Conclusion</b></h1>

<p style = 'font-size:16px;font-family:Arial'>In this demo we have done data preprocessing and created two commonly used predictive models for classification and predicted the customers that are likely to churn. We have also used AUC to get the accuracy of the generated models. 

<h1 style = 'font-size:28px;font-family:Arial;color:#E37C4D'><b>Cleanup</b></h1>

<p style = 'font-size:16px;font-family:Arial;color:#E37C4D'><b>Work Tables</b></p>
<p style = 'font-size:16px;font-family:Arial;'>
Cleanup work tables to prevent errors next time.

In [ ]:
tables = ['churn', 'onehotencodingfittable','ordinalencodingfittable','scaleFitOut','Transformed_data','xgb_model','xgb_predict_out']

# Loop through the list of tables and execute the drop table command for each table
for table in tables:
    try:
        db_drop_table(table_name = table)
    except:
        pass

<p style = 'font-size:16px;font-family:Arial;color:#E37C4D'><b>Databases and Tables</b></p>
<p style = 'font-size:16px;font-family:Arial'>The following code will clean up tables and databases created above.</p>

In [ ]:
%run -i ../run_procedure.py "call remove_data('DEMO_Telco');" 
#Takes 10 seconds

In [ ]:
remove_context()

<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>Dataset:</b>

- `CustomerID`: unique id of customer
- `Gender`: Whether the customer is a male or a female
- `SeniorCitizen`:Whether the customer is a senior citizen or not (1, 0)
- `Partner`:Whether the customer has a partner or not (Yes, No)
- `Dependents`:Whether the customer has dependents or not (Yes, No)
- `Tenure`:Number of months the customer has stayed with the company
- `PhoneService`:Whether the customer has a phone service or not (Yes, No)
- `MultipleLines`:Whether the customer has multiple lines or not (Yes, No, No phone service)
- `InternetService`:Customer’s internet service provider (DSL, Fiber optic, No)
- `OnlineSecurity`:Whether the customer has online security or not (Yes, No, No internet service)
- `OnlineBackup`:Whether the customer has online backup or not (Yes, No, No internet service)
- `DeviceProtection`:Whether the customer has device protection or not (Yes, No, No internet service)
- `TechSupport`:Whether the customer has tech support or not (Yes, No, No internet service)
- `StreamingTV`:Whether the customer has streaming TV or not (Yes, No, No internet service)
- `StreamingMovies`:Whether the customer has streaming movies or not (Yes, No, No internet service)
- `Contract`:The contract term of the customer (Month-to-month, One year, Two year)
- `PaperlessBilling`:Whether the customer has paperless billing or not (Yes, No)
- `PaymentMethod`:The customer’s payment method (Electronic check, Mailed check, Bank transfer (automatic), Credit card (automatic))
- `MonthlyCharges`:The amount charged to the customer monthly
- `TotalCharges`:The total amount charged to the customer
- `Churn`:Whether the customer churned or not (Yes or No)

<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>Reference Links:</b></p>
<ul style = 'font-size:16px;font-family:Arial'> 
       <li>Teradata Vantage™ - Analytics Database Analytic Functions - 17.20: <a href = 'https://docs.teradata.com/r/Enterprise_IntelliFlex_VMware/Teradata-VantageTM-Analytics-Database-Analytic-Functions-17.20/Introduction-to-Analytics-Database-Analytic-Functions '>https://docs.teradata.com/r/Enterprise_IntelliFlex_VMware/Teradata-VantageTM-Analytics-Database-Analytic-Functions-17.20/Introduction-to-Analytics-Database-Analytic-Functions </a></li>    
  <li>Teradata® Package for Python User Guide - 17.20: <a href = 'https://docs.teradata.com/r/Enterprise_IntelliFlex_VMware/Teradata-Package-for-Python-User-Guide-17.20/Introduction-to-Teradata-Package-for-Python'>https://docs.teradata.com/r/Enterprise_IntelliFlex_VMware/Teradata-Package-for-Python-User-Guide-17.20/Introduction-to-Teradata-Package-for-Python</a></li>
  <li>Teradata® Package for Python Function Reference - 17.20: <a href = 'https://docs.teradata.com/r/Enterprise/Teradata-Package-for-Python-Function-Reference-17.20/Teradata-Package-for-Python-Function-Reference'>https://docs.teradata.com/r/Enterprise/Teradata-Package-for-Python-Function-Reference-17.20/Teradata-Package-for-Python-Function-Reference</a></li>      
</ul>


<footer style="padding:10px;background:#f9f9f9;border-bottom:3px solid #394851">©2023 Teradata. All Rights Reserved</footer>